<a href="https://colab.research.google.com/github/fralfaro/MAT281/blob/main/docs/labs/lab_06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MAT281 - Laboratorio N°06


**Objetivo**: Aplicar técnicas básicas de **Machine Learning**, desde la preparación de datos hasta el entrenamiento y evaluación de modelos.




<p align="center">
  <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/e/ec/Anscombe%27s_quartet_3.svg/1200px-Anscombe%27s_quartet_3.svg.png" width="500"/>
</p>

El **cuarteto de Anscombe** es un ejemplo clásico en estadística que ilustra cómo diferentes conjuntos de datos pueden compartir las mismas propiedades estadísticas, como media, varianza y correlación, pero presentan comportamientos muy distintos cuando se visualizan gráficamente. Cada uno de los cuatro conjuntos consiste en once puntos (x, y) y fue creado por el estadístico F. J. Anscombe en 1973. Esta herramienta resalta la importancia de la visualización de datos para evitar interpretaciones erróneas basadas únicamente en análisis numéricos.

**Descripción del conjunto**

1. **Propiedades estadísticas comunes:** Todos los conjuntos tienen el mismo valor promedio para las variables \(x\) e \(y\), la misma varianza para \(x\) e \(y\), y una correlación lineal idéntica.
2. **Diferencias gráficas:** A pesar de sus similitudes estadísticas, los cuatro conjuntos presentan gráficos muy distintos:
   - El primer conjunto muestra una relación lineal simple.
   - El segundo conjunto tiene una relación no lineal, con una curva clara.
   - El tercer conjunto tiene una relación lineal clara, pero con un punto atípico que influye significativamente.
   - El cuarto conjunto tiene la mayoría de los puntos alineados verticalmente, con un punto atípico que afecta la correlación.

Este cuarteto enfatiza que las estadísticas descriptivas por sí solas pueden no capturar la esencia completa de los datos, subrayando la necesidad de utilizar visualizaciones en cualquier análisis exploratorio de datos.

In [ ]:
# Importar las bibliotecas necesarias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Configuración de los gráficos
%matplotlib inline
sns.set_theme(style="whitegrid")  # Establece un tema general para los gráficos
sns.set_palette("deep", desat=0.6)
plt.rcParams['figure.figsize'] = (12, 8)  # Ajuste del tamaño de las figuras

# Cargar los datos del cuarteto de Anscombe
data = sns.load_dataset("anscombe")

# Mostrar las primeras filas del conjunto de datos
data.head()

,dataset,x,y
0,I,10.0,8.04
1,I,8.0,6.95
2,I,13.0,7.58
3,I,9.0,8.81
4,I,11.0,8.33


Con base en la información presentada y el análisis realizado, les invitamos a reflexionar y responder las siguientes preguntas. Estas preguntas están diseñadas para profundizar en su comprensión del cuarteto de Anscombe y fomentar un análisis crítico de los datos:



1. Cree un gráfico de dispersión (scatter plot) para cada uno de los cuatro grupos del cuarteto de Anscombe. A partir de la visualización, ¿puede identificar diferencias significativas entre los grupos? ¿Qué características particulares observa en cada uno que sugieren comportamientos distintos?



In [ ]:
# FIXME
# 1) Gráficos de dispersión del cuarteto de Anscombe (uno por grupo)
import seaborn as sns
import matplotlib.pyplot as plt

sns.set(style="whitegrid")
g = sns.lmplot(
    data=data, x="x", y="y",
    col="dataset", col_wrap=2,
    ci=None, height=4,
    scatter_kws={"s": 50}, line_kws={"lw": 2}
)
g.set_titles("Dataset {col_name}")
g.fig.subplots_adjust(top=0.9)
g.fig.suptitle("Cuarteto de Anscombe: dispersión y ajuste lineal")

# Respuesta:
# I: relación lineal positiva clara; II: patrón curvo (no lineal);
# III: casi lineal con un outlier en X que distorsiona la recta;
# IV: puntos casi verticales en una misma x + un outlier que induce correlación aparente.
# por lo tanto pese a tener estadísticas iguales, los comportamientos son distintos mejor siempre visualizar los datos.


2. Utilice el comando `describe` para generar un resumen de las medidas estadísticas más relevantes para cada uno de los grupos del cuarteto de Anscombe. A partir de estos resultados, interprete las estadísticas obtenidas, destacando las características más significativas de cada grupo y cómo pueden influir en la comprensión de sus respectivas distribuciones.


In [ ]:
# FIXME
desc_stats = data.groupby("dataset").describe()
print(desc_stats)

# Respuesta:
# Todos los grupos tienen promedios, varianzas y correlaciones similares,
# pero estos valores no reflejan las diferencias estructurales observadas en los gráficos.
# El resumen estadístico es insuficiente para distinguir la verdadera forma de los datos.


3. Ajuste un modelo de regresión lineal para cada grupo utilizando **sklearn**. Calcule las métricas de evaluación, como el error cuadrático medio (MSE) y R², y grafique los resultados de la regresión. Interprete los resultados y su impacto en la calidad del ajuste.



In [ ]:
# FIXME
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

results = {}

for d in data["dataset"].unique():
    subset = data[data["dataset"] == d]
    X = subset[["x"]]
    y = subset["y"]

    model = LinearRegression().fit(X, y)
    y_pred = model.predict(X)

    mse = mean_squared_error(y, y_pred)
    r2 = r2_score(y, y_pred)

    results[d] = {
        "coef": model.coef_[0],
        "intercept": model.intercept_,
        "MSE": mse,
        "R2": r2
    }

    # Gráfico del ajuste
    plt.scatter(X, y, label="Datos")
    plt.plot(X, y_pred, color="red", label="Regresión")
    plt.title(f"Dataset {d} - Regresión lineal")
    plt.legend()
    plt.show()

print(results)

# Respuesta:
# En los cuatro grupos la regresión entrega parámetros y métricas similares (MSE y R²),
# pero al comparar con las gráficas se observa que el ajuste es bueno solo en el dataset I.
# En II la relación real es no lineal, en III un outlier distorsiona el modelo, y en IV
# un solo punto induce correlación sin representar el patrón de la mayoría de datos.



4. Es evidente que el ajuste lineal no es adecuado para algunos grupos. Existen diversas estrategias para abordar este problema, como eliminar outliers o emplear diferentes modelos de regresión. Identifique una estrategia que podría mejorar el ajuste del modelo de regresión lineal y, si lo considera necesario, implemente otros modelos alternativos para aquellos casos donde el ajuste lineal resulte inadecuado.

In [ ]:
# FIXME
# aplicar un modelo no lineal (polinómico) en el dataset II
import numpy as np
from sklearn.preprocessing import PolynomialFeatures

subset = data[data["dataset"] == 2]
X = subset[["x"]]
y = subset["y"]

# Transformación polinómica de grado 2
poly = PolynomialFeatures(degree=2)
X_poly = poly.fit_transform(X)

model_poly = LinearRegression().fit(X_poly, y)
y_pred_poly = model_poly.predict(X_poly)

plt.scatter(X, y, label="Datos")
plt.plot(X, y_pred_poly, color="green", label="Regresión polinómica (grado 2)")
plt.title("Dataset II - Ajuste no lineal")
plt.legend()
plt.show()

# Respuesta:
# Para datos como el conjunto II, un modelo polinómico captura mejor la curva.
# En el conjunto III sería adecuado detectar y eliminar/eliminar outliers.
# En el conjunto IV, la correlación artificial obliga a replantear el uso de regresión lineal,
# considerando modelos más robustos o análisis alternativos.
